In [21]:
import csv
import codecs
import collections
import nltk
import pandas as pd
import numpy as np

tokenizer = nltk.tokenize.TweetTokenizer()

In [2]:
df = pd.read_csv('D:\Max\docs_cluster\lenta\lenta_data.csv')
text_tags = df['tags'].tolist()
tags = sorted(set(text_tags))
texts = df['text'].tolist()


from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(tags)
text_tags = le.transform(text_tags)

In [8]:
corpus_size = len(texts)
selections_num = 6      # На сколько частей разбиваем датасет
selection_size = corpus_size // selections_num
print(selection_size)
print(corpus_size - (selections_num - 1) * selection_size)


for i in range(selections_num - 1):
    start = i * selection_size
    end = start + selection_size
    print(str(start) + ' - ' + str(end))

105936
105937
0 - 105936
105936 - 211872
211872 - 317808
317808 - 423744
423744 - 529680


Тренируемся на первых пяти датасетах без лемматизации, проверяемся на шестом

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline


lin_reg_clf = Pipeline([('vect', CountVectorizer()), 
             ('tfidf', TfidfTransformer()),
             ('clf', LogisticRegression())
               ])
                
lin_reg_models = []

for i in range(selections_num - 1):
    print("Selection #" + str(i))
    start = i * selection_size
    end = start + selection_size
    print("Training logistic regression")
    lin_reg_models.append(lin_reg_clf.fit(texts[start:end], text_tags[start:end]))

Selection #0
Training logistic regression
Selection #1
Training logistic regression
Selection #2
Training logistic regression
Selection #3
Training logistic regression
Selection #4
Training logistic regression


In [24]:
from sklearn.metrics import accuracy_score


start = (selections_num - 1) * selection_size
test_selection_cnt = corpus_size - (selections_num - 1) * selection_size
test_tags = text_tags[start:]

'''
X_test_counts = count_vect.fit(texts[start:])
X_test_tfidf = tfidf_transformer.fit(X_test_counts)
'''

lin_reg_predictions = [model.predict(texts[start:]) for model in lin_reg_models]
prediction = []
#multinom_predictions = [model.predict(X_test_tfidf) for model in multinom_models
for i in range(len(lin_reg_predictions[0])):
    predictions = [p[i] for p in lin_reg_predictions]
    prediction.append(max(set(predictions), key=predictions.count))


prediction = np.array(prediction)
log_reg_accuracy = accuracy_score(test_tags, prediction)
print(log_reg_accuracy)

0.791555358373


# Ниже черновик

In [44]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, twenty_train.target)

In [7]:
import pandas as pd
df = pd.read_csv('D:\Max\docs_cluster\lenta\lenta_data.csv')

In [13]:
df[:3]

,tags,text,title,topic,url
0,Деловой климат,Заместитель председателя правительства Аркадий...,Правительство прокомментировало идею о запрете...,Бизнес,https://lenta.ru/news/2017/04/01/24hours/
1,События,Монреальская конвенция об унификации правил во...,Эксперт отвел год на окончательную ратификацию...,Путешествия,https://lenta.ru/news/2017/03/31/motrealwork/
2,Общество,Сотни байкеров в столице Аргентины Буэнос-Айре...,Аргентинские байкеры устроили акцию протеста п...,Мир,https://lenta.ru/news/2017/03/30/bikers/


In [3]:
# ['tags', 'text', 'title', 'topic', 'url']
tags = [' спецпроект',
 'Coцсети',
 'Авто',
 'Автобизнес',
 'Аналитика рынка',
 'Архитектура',
 'Банки',
 'Баскетбол',
 'Безопасность ',
 'Белоруссия',
 'Бокс и ММА',
 'Вещи',
 'Вирусные ролики',
 'Вкусы',
 'Внешний вид',
 'Вооружение',
 'Вооруженные силы',
 'Все',
 'Выборы',
 'Гаджеты',
 'Госрегулирование',
 'Госэкономика',
 'Движение',
 'Деловой климат',
 'Деньги',
 'Достижения',
 'Еда',
 'Звери',
 'Зимние виды',
 'Игры',
 'Инновации',
 'Инструменты',
 'Интернет',
 'Искусство',
 'История',
 'Кавказ',
 'Казахстан',
 'Катастрофы',
 'Кино',
 'Книги',
 'Компании',
 'Конфликты',
 'Космос',
 'Криминал',
 'Крым',
 'Культпросвет',
 'Легпром',
 'Летние виды',
 'Люди',
 'Мемы',
 'Мир',
 'Мировой бизнес',
 'Мировой опыт',
 'Мнения',
 'Молдавия',
 'Москва',
 'Музыка',
 'Наследие',
 'Наука',
 'Общество',
 'Оружие',
 'Первая мировая',
 'Политика',
 'Полиция и спецслужбы',
 'Пресса',
 'Преступность',
 'Прибалтика',
 'Производители',
 'Происшествия',
 'Регионы',
 'Реклама',
 'Ресурсы',
 'Россия',
 'Рынки',
 'Следствие и суд',
 'События',
 'Средняя Азия',
 'Стиль',
 'Страноведение',
 'ТВ и радио',
 'Театр',
 'Теннис',
 'Техника',
 'Технологии',
 'Туризм',
 'Украина',
 'Финансы компаний',
 'Футбол',
 'Хоккей',
 'Часы',
 'Экология',
 'Явления']

num_tags = [{'tag': tag, 'id': i} for i, tag in enumerate(tags)]

texts = []
text_tags = []

with codecs.open('D:\Max\docs_cluster\lenta\lenta_data.csv', 'r', 'utf_8_sig') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        if i == 0:
            continue
        
        texts.append(row[1])
        text_tags.append(next((tag['id'] for tag in num_tags if tag['tag'] == row[0]), None))